In [0]:
import pandas as pd
from datetime import datetime,timedelta
from google.colab import files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Cleaning Movie List

In [0]:
df=pd.read_excel("/content/Movie_dataset_3subcategories.xlsx",sheet_name="All years combined")
#df_2019=df[df['Year']==2019][['Year','Opening','Date','Title','Unnamed: 4','Director','Cast','Studio (production house)','Subcategories']]
df_2019=df[['Year','Opening','Date','Title','Unnamed: 4','Director','Cast','Studio (production house)','Subcategories 1','Subcategories 2','Subcategories 3']]
df_2019=df_2019.rename(columns={"Unnamed: 4":"uni_code","Studio (production house)":"Studio"})
df_2019=df_2019[df_2019['uni_code']!=""]

In [0]:
df_2019.head()

,Year,Opening,Date,Title,uni_code,Director,Cast,Studio,Subcategories 1,Subcategories 2,Subcategories 3
0,2018,October,5,Andhadhun,2iVYI99VGaw,Sriram Raghavan,"Ayushman Khurana, Tabu, Radhika Apte",Viacom 18 Motion Pictures,4,4,3
1,2019,March,8,Badla,mSlgu8AQAd4,Sujoy Ghosh,"Amitabh Bachchan, Taapsee Pannu","Red Chillies Entertainment, Azure Entertainment",4,4,3
2,2019,January,11,Uri: The Surgical Strike,VVY3do673Zc,Aditya Dhar,"Vicky Kaushal, Mohit Raina, Paresh Rawal, Yami...",RSVP Movies,4,4,3
3,2018,March,23,Hichki,nLSaCFlXn-g,Siddharth P Malhotra,Rani Mukerji,Yash Raj Films,4,4,3
4,2018,February,9,Pad Man,-K9ujx8vO_A,R. Balki,"Akshay Kumar, Sonam Kapoor, Radhika Apte","Columbia Pictures KriArj Entertainment, Hope P...",4,4,3


In [0]:
#Creating the threshold datettime

df_2019['Month']=[datetime.strptime(x,"%B").month for x in df_2019['Opening']]

df_2019['Month']=df_2019['Month'].astype(str)
df_2019['Month']=["0"+ x if len(x)==1 else x for x in df_2019['Month']]
df_2019['Date']=df_2019['Date'].astype(str)
df_2019['Date']=["0"+ x if len(x)==1 else x for x in df_2019['Date']]

df_2019['Datef']=df_2019['Year'].astype(str)+"-"+df_2019['Month'].astype(str)+"-"+df_2019['Date'].astype(str)

df_2019['Datef']=[datetime.strptime(x,"%Y-%m-%d") for x in df_2019['Datef']]
df_2019['Threshold']=[x-timedelta(days=7) for x in df_2019['Datef']]
df_2019['lower_thres']=[x-timedelta(days=21) for x in df_2019['Datef']]
df_2019['Week']=[x.isocalendar()[1] for x in df_2019['Datef']]
del df_2019['Datef']
del df_2019['Month']
del df_2019['Year']

df_2019=df_2019.fillna("")

In [0]:
#parsing cast to list
df_2019['Cast']=[x+", " for x in df_2019['Cast']]
df_2019['Cast 1']=[x.split(",")[0] for x in df_2019['Cast']]
df_2019['Cast 1']=[x.lower() for x in df_2019['Cast 1']]
df_2019['Cast 1']=[x.strip() for x in df_2019['Cast 1']]

df_2019['Cast 2']=[x.split(",")[1] for x in df_2019['Cast']]
df_2019['Cast 2']=[x.lower() for x in df_2019['Cast 2']]
df_2019['Cast 2']=[x.strip() for x in df_2019['Cast 2']]

"""
#parsing studio name to list
df_2019['Studio']=[x.split(",") for x in df_2019['Studio']]
l_new=[]
for x in df_2019['Studio']:
  l_new.append([p.lstrip() for p in x])

df_2019['Studio']=l_new

#parsing directors to list
df_2019['Director']=[x.split(",") for x in df_2019['Director']]
l_new=[]
for x in df_2019['Director']:
  l_new.append([p.lstrip() for p in x])

df_2019['Director']=l_new
"""

'\n#parsing studio name to list\ndf_2019[\'Studio\']=[x.split(",") for x in df_2019[\'Studio\']]\nl_new=[]\nfor x in df_2019[\'Studio\']:\n  l_new.append([p.lstrip() for p in x])\n\ndf_2019[\'Studio\']=l_new\n\n#parsing directors to list\ndf_2019[\'Director\']=[x.split(",") for x in df_2019[\'Director\']]\nl_new=[]\nfor x in df_2019[\'Director\']:\n  l_new.append([p.lstrip() for p in x])\n\ndf_2019[\'Director\']=l_new\n'

In [0]:
df_2019.to_csv("all_celeaned_f4.csv")
files.download("all_celeaned_f4.csv")

##Developing Features

In [0]:
#installing required packages
pip install emot

In [0]:
#importing required libraries
import nltk.data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

import pandas as pd
import numpy as np
import scipy as sp
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import words
from itertools import combinations
import random as rd
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
#from googletrans import Translator
from datetime import datetime
from nltk.stem import WordNetLemmatizer 
import ast
from scipy import sparse

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [0]:
parent_list=df_2019.copy()
L_zip =list(zip(parent_list['uni_code'],parent_list['Threshold'],parent_list['lower_thres']))

In [0]:
L_zip[0:10]

[('2iVYI99VGaw',
  Timestamp('2018-09-28 00:00:00'),
  Timestamp('2018-09-14 00:00:00')),
 ('mSlgu8AQAd4',
  Timestamp('2019-03-01 00:00:00'),
  Timestamp('2019-02-15 00:00:00')),
 ('VVY3do673Zc',
  Timestamp('2019-01-04 00:00:00'),
  Timestamp('2018-12-21 00:00:00')),
 ('nLSaCFlXn-g',
  Timestamp('2018-03-16 00:00:00'),
  Timestamp('2018-03-02 00:00:00')),
 ('-K9ujx8vO_A',
  Timestamp('2018-02-02 00:00:00'),
  Timestamp('2018-01-19 00:00:00')),
 ('x_7YlGv9u1g',
  Timestamp('2016-12-16 00:00:00'),
  Timestamp('2016-12-02 00:00:00')),
 ('unAljCZMQYw',
  Timestamp('2018-10-11 00:00:00'),
  Timestamp('2018-09-27 00:00:00')),
 ('q10nfS9V090',
  Timestamp('2019-08-08 00:00:00'),
  Timestamp('2019-07-25 00:00:00')),
 ('vb5xCMbMfZ0',
  Timestamp('2016-01-15 00:00:00'),
  Timestamp('2016-01-01 00:00:00')),
 ('veJ6ejMjzgE',
  Timestamp('2019-10-31 00:00:00'),
  Timestamp('2019-10-17 00:00:00'))]

### Utility Functions

In [0]:
#Declaring all necessary functions
setofwords = set(words.words())

#Convert emojis to text
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, " " + "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
        #text = text.replace(emot,"")
    return text

#apply emoji convert & translate hindi to english
def convert_lang(x,date1,date2): 
  path="/content/drive/My Drive/project/allmovies/"+x+".csv"
  try:
    df=pd.read_csv(path,engine='python')
  except:
    df=pd.read_csv(path,engine='c')
  del df['Unnamed: 0']
  df=df.dropna()
  df['pub_dates']=[datetime.strptime(x.split("T")[0], "%Y-%m-%d") for x in df['pub_dates']]
  comments=df[(df['pub_dates']<date1) & (df['pub_dates']>date2)]['comments']
  comments=[convert_emojis(x) for x in comments]
  return comments

#remove punctuations, special characters and stop words from the text
def clean_comments(comment):
  # removing all non aplhabetic characters
  clean_comment=re.sub("[^a-zA-Z]+"," ",comment)

  #lowercase & split
  clean_comment = clean_comment.lower()
  clean_comment = clean_comment.split()

  #remove stop words & join
  from nltk.corpus import stopwords
  clean_comment = [w for w in clean_comment if not w in stopwords.words("english")]
  lemmatizer = WordNetLemmatizer()
  clean_comment=[lemmatizer.lemmatize(w) for w in clean_comment]
  clean_comment=[x for x in clean_comment if x in setofwords]
  clean_comment=" ".join( clean_comment)

  return clean_comment

#get words that have highest tfidf score
def get_topwords(cleaned_comments):
  print("getting_top_words")
  cv=CountVectorizer(max_features=5000)
  word_count_vector=cv.fit_transform(cleaned_comments)
  tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
  tfidf_transformer.fit(word_count_vector)
  df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
  df_idf=df_idf.sort_values(by=["idf_weights"],ascending=False).iloc[0:200]
  return list(df_idf.index)

#### Creating top 200 words for all movies combined

In [0]:
final_vectors = pd.DataFrame(columns=['videoId','top_words'])
L_ex=[]
i=0
cleaned_comm=[]
for x,date1,date2 in L_zip:
  print(i)
  try:
    proc_comments = convert_lang(x,date1,date2)
    print("cleaning now")
    cleaned_comments=[clean_comments(comment) for comment in proc_comments]
    cleaned_comm=cleaned_comm+[".".join(cleaned_comments)]
    i=i+1
  except:
    print("excepted")
    print(i)
    L_ex=L_ex+[i]
    i=i+1
    continue
  

vectorizer = CountVectorizer(analyzer = "word",tokenizer = None, preprocessor = None, stop_words = None,max_features = 200)
train_data_features = vectorizer.fit_transform(cleaned_comm)
X_counts=train_data_features.toarray()
transformer=TfidfTransformer(smooth_idf=False)
tfidf=transformer.fit_transform(X_counts)
X_tfidf=tfidf.toarray()
X_tfidf2=np.where(X_tfidf>0,1,X_tfidf)

0
cleaning now
1
cleaning now
2
cleaning now
3
cleaning now
4
cleaning now
5
cleaning now
6
cleaning now
7
cleaning now
8
cleaning now
9
cleaning now
10
cleaning now
11
cleaning now
12
cleaning now
13
cleaning now
14
cleaning now
15
cleaning now
16
cleaning now
17
cleaning now
18
cleaning now
19
cleaning now
20
cleaning now
21
cleaning now
22
cleaning now
23
cleaning now
24
cleaning now
25
cleaning now
26
cleaning now
27
cleaning now
28
cleaning now
29
cleaning now
30
cleaning now
31
cleaning now
32
cleaning now
33
cleaning now
34
cleaning now
35
cleaning now
36
cleaning now
37
cleaning now
38
cleaning now
39
cleaning now
40
cleaning now
41
cleaning now
42
cleaning now
43
cleaning now
44
cleaning now
45
cleaning now
46
cleaning now
47
cleaning now
48
cleaning now
49
cleaning now
50
cleaning now
51
cleaning now
52
cleaning now
53
cleaning now
54
cleaning now
55
cleaning now
56
cleaning now
57
cleaning now
58
cleaning now
59
cleaning now
60
cleaning now
61
cleaning now
62
cleaning now
63

In [0]:
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None, preprocessor = None, stop_words = None,max_features = 700)
train_data_features = vectorizer.fit_transform(cleaned_comm)
X_counts=train_data_features.toarray()
transformer=TfidfTransformer(smooth_idf=False)
tfidf=transformer.fit_transform(X_counts)
X_tfidf=tfidf.toarray()
X_tfidf2=np.where(X_tfidf>0,1,X_tfidf)

In [0]:
dump=pd.DataFrame()
dump['dumpf4']=cleaned_comm
dump.to_csv("dump_f4_final.csv")
#files.download("dump_f4.csv")

In [0]:
files.download("dump_f4_final.csv")

In [0]:
f=open('f1.txt','w')
for ele in L_ex:
    f.write(str(ele)+'\n')

f.close()

In [0]:
files.download("f1.txt
")

In [0]:
"""
dump=pd.read_csv("/content/dump_f4 (1).csv")
dump=dump.fillna("")
cleaned_comm=list(dump['dumpf4'])
"""

In [0]:
for_tensor=df_2019.copy()
for_tensor=for_tensor.drop(index=L_ex)
#the following are needed when read from csv directly
#for_tensor['Cast']=[ast.literal_eval(x) for x in for_tensor['Cast']]
#for_tensor['Studio']=[ast.literal_eval(x) for x in for_tensor['Studio']]
#for_tensor['Director']=[ast.literal_eval(x) for x in for_tensor['Director']]
for_tensor.head()

,Opening,Date,Title,uni_code,Director,Cast,Studio,Subcategories 1,Subcategories 2,Subcategories 3,Threshold,lower_thres,Week,Cast 1,Cast 2
0,October,05,Andhadhun,2iVYI99VGaw,Sriram Raghavan,"Ayushman Khurana, Tabu, Radhika Apte,",Viacom 18 Motion Pictures,4,4,3,2018-09-28,2018-09-14,40,ayushman khurana,tabu
1,March,08,Badla,mSlgu8AQAd4,Sujoy Ghosh,"Amitabh Bachchan, Taapsee Pannu,","Red Chillies Entertainment, Azure Entertainment",4,4,3,2019-03-01,2019-02-15,10,amitabh bachchan,taapsee pannu
2,January,11,Uri: The Surgical Strike,VVY3do673Zc,Aditya Dhar,"Vicky Kaushal, Mohit Raina, Paresh Rawal, Yami...",RSVP Movies,4,4,3,2019-01-04,2018-12-21,2,vicky kaushal,mohit raina
3,March,23,Hichki,nLSaCFlXn-g,Siddharth P Malhotra,"Rani Mukerji,",Yash Raj Films,4,4,3,2018-03-16,2018-03-02,12,rani mukerji,
4,February,09,Pad Man,-K9ujx8vO_A,R. Balki,"Akshay Kumar, Sonam Kapoor, Radhika Apte,","Columbia Pictures KriArj Entertainment, Hope P...",4,4,3,2018-02-02,2018-01-19,6,akshay kumar,sonam kapoor


In [0]:
df_cast=pd.read_csv("/content/actor-cat.csv")

In [0]:
keys=list(df_cast['Cast'])
keys=keys+[""]
vals=list(df_cast['category'])
vals=vals+[0]
dict_cov=dict(zip(keys,vals))

In [0]:
for_t2=for_tensor.copy()

In [0]:
for_t2['Cast 1s']=[dict_cov[x] for x in for_t2['Cast 1']]
for_t2['Cast 2s']=[dict_cov[x] for x in for_t2['Cast 2']]
for_t2.head()
for_t3=for_t2[['Title', 'uni_code']]

In [0]:
d={1: [0]*len(for_t2),2: [0]*len(for_t2),3: [0]*len(for_t2),4: [0]*len(for_t2),0: [0]*len(for_t2)}

#print(d)
for i in range(0,len(for_t2)):
  #print(i)
  k1=for_t2['Cast 1s'].iloc[i]
  k2=for_t2['Cast 2s'].iloc[i]
  #print(d[k1][i])
  d[k1][i]=d[k1][i]+1
  d[k2][i]=d[k2][i]+1
  #d[for_t2['Cast 2s'].iloc[i]][i]=d[for_t2['Cast 2s'].iloc[i]][i]+1
  #print(d)


for_t3['a_cat1']=d[1]
for_t3['a_cat2']=d[2]
for_t3['a_cat3']=d[3]
for_t3['a_cat4']=d[4]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [0]:
for_t2.head()

,Opening,Date,Title,uni_code,Director,Cast,Studio,Subcategories 1,Subcategories 2,Subcategories 3,Threshold,lower_thres,Week,Cast 1,Cast 2,Cast 1s,Cast 2s,a_cat1,a_cat2,a_cat3,a_cat4
0,October,05,Andhadhun,2iVYI99VGaw,Sriram Raghavan,"Ayushman Khurana, Tabu, Radhika Apte,",Viacom 18 Motion Pictures,4,4,3,2018-09-28,2018-09-14,40,ayushman khurana,tabu,2,4,0,1,0,1
1,March,08,Badla,mSlgu8AQAd4,Sujoy Ghosh,"Amitabh Bachchan, Taapsee Pannu,","Red Chillies Entertainment, Azure Entertainment",4,4,3,2019-03-01,2019-02-15,10,amitabh bachchan,taapsee pannu,1,2,1,1,0,0
2,January,11,Uri: The Surgical Strike,VVY3do673Zc,Aditya Dhar,"Vicky Kaushal, Mohit Raina, Paresh Rawal, Yami...",RSVP Movies,4,4,3,2019-01-04,2018-12-21,2,vicky kaushal,mohit raina,3,4,0,0,1,1
3,March,23,Hichki,nLSaCFlXn-g,Siddharth P Malhotra,"Rani Mukerji,",Yash Raj Films,4,4,3,2018-03-16,2018-03-02,12,rani mukerji,,2,0,0,1,0,0
4,February,09,Pad Man,-K9ujx8vO_A,R. Balki,"Akshay Kumar, Sonam Kapoor, Radhika Apte,","Columbia Pictures KriArj Entertainment, Hope P...",4,4,3,2018-02-02,2018-01-19,6,akshay kumar,sonam kapoor,1,1,2,0,0,0


In [0]:
d_cal={}
for i in range(1,53):
  d_cal.update({i:[0]*len(for_t2)})

In [0]:
for i in range(0,len(for_t2)):
  #print(i)
  k=for_t2['Week'].iloc[i]
  #print(d[k1][i])
  d_cal[k][i]=d_cal[k][i]+1
  #d[k2][i]=d[k2][i]+1
  #d[for_t2['Cast 2s'].iloc[i]][i]=d[for_t2['Cast 2s'].iloc[i]][i]+1
  #print(d)

for i in range(1,53):
  name="Week "+str(i)
  for_t3[name]=d_cal[i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [0]:
for_t3.head()
del for_t3['Title']
del for_t3['uni_code']

In [0]:
for_tensor['Cast 1']=[dict_cov[x] for x in for_tensor['Cast 1']]
for_tensor['Cast 2']=[dict_cov[x] for x in for_tensor['Cast 2']]
for_tensor['Cast score']=for_tensor['Cast 1']+for_tensor['Cast 2']

In [0]:
for_mat=for_tensor[['Cast score','Week']]
for_mat=for_mat.to_numpy()

In [0]:
for_mat2=for_t3.to_numpy()

In [0]:
s_t=np.concatenate((X_tfidf2,for_mat2),axis=1)

In [0]:
X_tfidf2.shape

(420, 700)

In [0]:
cat_result1 = np.array(for_tensor['Subcategories 1'])
cat_result2 = np.array(for_tensor['Subcategories 2'])
cat_result3 = np.array(for_tensor['Subcategories 3'])
#org_mat = mat.toarray()
org_mat_df = pd.DataFrame(data=s_t, columns=['c'+str(i) for i in range(s_t.shape[1])])
org_mat_df.to_csv("/content/drive/My Drive/project/Results/F4_model_data_700_catex.csv")
cat_result_df1 = pd.DataFrame({'t': cat_result1})
cat_result_df1.to_csv("/content/drive/My Drive/project/Results/F4_model_data_target_1.csv")
cat_result_df2 = pd.DataFrame({'t': cat_result2})
cat_result_df2.to_csv("/content/drive/My Drive/project/Results/F4_model_data_target_2.csv")
cat_result_df3 = pd.DataFrame({'t': cat_result3})
cat_result_df3.to_csv("/content/drive/My Drive/project/Results/F4_model_data_target_3.csv")